In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import statsmodels.api as sm
import scipy.stats as stats

In [3]:
#############
### files ###
#############

file1 = 'GPEX1set1.csv'
file2 = 'GPEX1set2.csv'
file3 = 'GPEX1set3.csv'

In [5]:
####################################
### define some useful functions ###
####################################

# winsorization given one variable
def winsorize(var, lower, upper):

    """
    Parameters:
    var : the variable to be winsorized
    lower : lower percentile, e.g., 0.01
    upper : upper percentile, e.g., 0.99
    """
    
    s = var.quantile([lower, upper])
    l = s.iloc[0]
    u = s.iloc[-1]
    var_win = var.clip(l, u)

    return var_win

# winsorization given mutiple variables
def winsorize_n(df, lower, upper, *args):
    
    for col in args:
        df[col] = winsorize(df[col], lower, upper)

    return df

# OLS regression
def lm(df, y, *args):

    # fit the model
    y = df[y].to_numpy()
    X = sm.add_constant(df[list(args)], prepend=True).to_numpy()
    model = sm.OLS(y, X).fit()

    # get the required values
    coef = model.params
    r2_adj = model.rsquared_adj
    n = model.nobs

    # organize them
    cols = dict(zip(['constant'] + list(args), coef))
    cols['adj_R^2'] = r2_adj
    cols['#obs'] = n
    out = pd.DataFrame(cols, index=[0], columns=['constant'] + list(args) + ['adj_R^2', '#obs'])
    
    return out

# calculate time-series avg, std_error,t-stat ,p-value
def tstat(df):
    
    # mean
    avg = df.mean(axis=0)    
    # std_error
    sem = df.sem(axis=0)    
    # t-stat
    t = avg / sem
    # row count
    rowc = df.shape[0]
    # number of parameters estimated (2 parameters estimated: average and standard error)
    n = 2
    # degree of freedom
    dof = rowc - n
    
    res = pd.concat([avg,t], axis=1, keys=['average','t-stat'])

    # calculate t-values
    def tstat_helper(res,dof):
        
        alpha_1 = 0.01
        alpha_5 = 0.05
        alpha_10 = 0.10
    
        t_value1 = stats.t.ppf(1 - alpha_1 / 2, dof) # two tailed test
        t_value2 = stats.t.ppf(1 - alpha_5 / 2, dof) # two tailed test
        t_value3 = stats.t.ppf(1 - alpha_10 / 2, dof) # two tailed test
            
        if abs(res['t-stat'])>t_value1:
            return '***'
        elif abs(res['t-stat'])>t_value2:
            return '**'
        elif abs(res['t-stat'])>t_value3:
            return '*'
        else:
            return 'not sig.'

    res['t-test'] = res.apply(tstat_helper ,dof=dof, axis=1)

    return res

# forecast
def forecast(df, *args):

    """
    If firm characteristics are denoted by 'c', then this func assumes the corresponding risk premia
    are denoted by 'c_lambda'
    """

    fcst = df['constant']
    for c in args:
        fcst += df[c] * df[f'{c}_lambda']

    res = (
        df.pipe(pd.DataFrame.assign,
                forecast=fcst,
               )
        .pipe(pd.DataFrame.dropna, subset=['forecast'])
        .pipe(pd.DataFrame.reindex, columns=['date', 'gvkey', 'return', 'forecast'])
        .pipe(pd.DataFrame.reset_index, drop=True)
    )
    
    return res

# forecasting accuracy based on the sign
def accuracy(df, target, predicted):

    """
    target and predicted are the names of the corresponding columns
    """
    
    res = (
        df.pipe(pd.DataFrame.assign,
                comp=lambda x: np.where(np.sign(x[target]) == np.sign(x[predicted]), 1, 0)
               )
        .pipe(pd.DataFrame.reindex, columns=['date', 'comp'])
        .pipe(lambda x: x.groupby('date')['comp'].mean().reset_index().rename(columns={'comp': 'accuracy'}))
    )

    return res

# performance by every month
def performance(df, target, predicted):

    res = (
        df.pipe(pd.DataFrame.assign,
                ls=lambda x: x[target].where(x[predicted] > 0, -x[target])
               )
        .pipe(pd.DataFrame.reindex, columns=['date', 'ls'])
        .pipe(lambda x: x.groupby('date')['ls'].mean().reset_index().rename(columns={'ls': 'mret'}))
    )
        
    return res

# annualized return
def annualret(df):

    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    # Change return from % to decimal
    df['mret'] /=100
    df['mret'] +=1

    res = (df.groupby('year')['mret'].agg(lambda x: (x.prod() - 1)*100)
        .reset_index().rename(columns={'year': 'Year', 'mret': 'Annualised Return(%)'})
          )

    return res


# Question 1

Fama-MacBeth Regression

## Model 1

In [10]:
# read the dataset
clct1 = []
rdr1 = pd.read_csv(file1, dtype={'GVKEY': str}, parse_dates=[1], chunksize=100000)
for chunk in tqdm(rdr1):
    chunk = chunk.rename(str.lower, axis=1)
    clct1.append(chunk)
    chunk = None
md1 = pd.concat(clct1, ignore_index=True)
clct1 = None
md1.head()


23it [00:03,  6.57it/s]


,gvkey,date,return,logsize_-1,logb/m_-1,"return_-2,-12"
0,1000,1972-04-30,26.6667,2.819480,-0.266121,-0.461539
1,1000,1972-05-31,-7.0175,3.053069,-0.934455,-0.476744
2,1000,1972-06-30,-9.4340,2.977503,-0.861695,-0.173913
3,1000,1972-07-31,0.0000,2.875596,-0.762604,-0.196970
4,1000,1972-08-31,-2.0833,2.875935,-0.762604,-0.020408


In [12]:
# Preprocessing

# Extract the list of predictors
predictor = tuple(md1.columns[3:]) 

# Keep the list of months for regression only if the #stocks returns for that month >= 30
month_list = (
    md1.groupby('date').size().reset_index()
    .pipe(pd.DataFrame.rename, columns={0: 'n_stocks'})
    .pipe(pd.DataFrame.query, "n_stocks >= 30")
    .pipe(lambda x: x['date'].to_list())
)

md1 = (
    md1[md1['date'].isin(month_list)]
)
md1.head()

,gvkey,date,return,logsize_-1,logb/m_-1,"return_-2,-12"
0,1000,1972-04-30,26.6667,2.819480,-0.266121,-0.461539
1,1000,1972-05-31,-7.0175,3.053069,-0.934455,-0.476744
2,1000,1972-06-30,-9.4340,2.977503,-0.861695,-0.173913
3,1000,1972-07-31,0.0000,2.875596,-0.762604,-0.196970
4,1000,1972-08-31,-2.0833,2.875935,-0.762604,-0.020408


In [14]:
# Fama-MacBeth reg
tb1 = md1.groupby('date').apply(lm, 'return', *predictor).droplevel(-1, axis=0)
#tb1.head(10)
print(pd.concat([tb1.head(5),tb1.tail(5)]))

             constant  logsize_-1  logb/m_-1  return_-2,-12   adj_R^2    #obs
date                                                                         
1967-04-30  -0.738475   -0.043935   0.483928      12.706879  0.260407    39.0
1967-05-31  -3.699470    0.152711  -0.786294     -14.648477  0.169260    41.0
1967-06-30  16.817477   -2.709165  -0.209950       2.726668  0.052855    42.0
1967-07-31  23.554904   -2.546990   3.808583       6.305582  0.126701    42.0
1967-08-31   7.588720   -1.062317   3.229335      -1.055159  0.005256    42.0
2020-11-30  28.855404   -1.202329   1.751635      -4.558963  0.032880  3500.0
2020-12-31  16.383572   -1.281029  -1.029362      -1.544643  0.013615  3468.0
2021-01-31  22.089914   -1.622395   6.724511      29.714248  0.042235  1173.0
2021-02-28  19.432621   -0.797685   3.926728       1.939198  0.061897  1004.0
2021-03-31   7.015325    0.147179   2.026372      -1.332766  0.046385   923.0


/var/folders/cy/c49pzcm15cs4xg2nb2d36pkr0000gn/T/ipykernel_13579/1613005460.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tb1 = md1.groupby('date').apply(lm, 'return', *predictor).droplevel(-1, axis=0)


## Model 2

In [7]:
# read the dataset
clct2 = []
rdr2 = pd.read_csv(file2, dtype={'GVKEY': str}, parse_dates=[1], chunksize=100000)
for chunk in tqdm(rdr2):
    chunk = chunk.rename(str.lower, axis=1)
    clct2.append(chunk)
    chunk = None
md2 = pd.concat(clct2, ignore_index=True)
clct2 = None
md2.head()

15it [00:02,  5.62it/s]


,gvkey,date,return,logsize_-1,logb/m_-1,"return_-2,-12","logissues_-1,-36",accruals_yr-1,roa_yr-1,logag_yr-1
0,1000,1974-04-30,9.5238,1.924431,-0.081561,-0.428572,-0.298542,-0.246,0.0894,0.089507
1,1000,1974-05-31,-13.0435,2.016424,0.048045,-0.222223,-0.297521,-0.246,0.0894,0.089507
2,1000,1974-06-30,5.0000,1.877682,0.187806,-0.080001,-0.296501,-0.246,0.0894,0.089507
3,1000,1974-07-31,-14.2857,1.927492,0.139016,-0.166668,-0.295198,-0.246,0.0894,0.089507
4,1000,1974-08-31,16.6667,1.766826,0.293167,-0.322582,-0.301428,-0.246,0.0894,0.089507


In [8]:
# preprocessing
# the predictors are winsorized at 0.01 and 0.99 levels
# run cross-sectional reg for every month only the #stocks >= 30
predictor = tuple(md2.columns[3:])
month_list = (
    md2.groupby('date').size().reset_index()
    .pipe(pd.DataFrame.rename, columns={0: 'n_stocks'})
    .pipe(pd.DataFrame.query, "n_stocks >= 30")
    .pipe(lambda x: x['date'].to_list())
)

md2 = (
    md2[md2['date'].isin(month_list)]
)
md2.head()

,gvkey,date,return,logsize_-1,logb/m_-1,"return_-2,-12","logissues_-1,-36",accruals_yr-1,roa_yr-1,logag_yr-1
0,1000,1974-04-30,9.5238,1.924431,-0.081561,-0.428572,-0.298542,-0.246,0.0894,0.089507
1,1000,1974-05-31,-13.0435,2.016424,0.048045,-0.222223,-0.297521,-0.246,0.0894,0.089507
2,1000,1974-06-30,5.0000,1.877682,0.187806,-0.080001,-0.296501,-0.246,0.0894,0.089507
3,1000,1974-07-31,-14.2857,1.927492,0.139016,-0.166668,-0.295198,-0.246,0.0894,0.089507
4,1000,1974-08-31,16.6667,1.766826,0.293167,-0.322582,-0.301428,-0.246,0.0894,0.089507


In [9]:
# Fama-MacBeth reg
tb2 = md2.groupby('date').apply(lm, 'return', *predictor).droplevel(-1, axis=0)
#tb2.head(10)
print(pd.concat([tb2.head(5),tb2.tail(5)]))

             constant  logsize_-1  logb/m_-1  return_-2,-12  logissues_-1,-36  \
date                                                                            
1969-09-30  -4.867520   -0.815177  -6.928825       1.889119          1.162051   
1969-10-31   9.872629   -1.919749  -7.038207       2.836895          1.116628   
1969-11-30   0.329359   -0.581137   1.364054       0.150551         -6.569321   
1969-12-31 -11.956618    2.672885   2.111418      23.254293         -0.848763   
1970-01-31  -3.813598    0.980359   2.424982       2.928329          0.940534   
2020-11-30  25.473357   -0.520317   3.872598      -6.232116         -0.323231   
2020-12-31  18.356971   -1.517659  -0.907680      -1.545756         -1.157680   
2021-01-31  19.685829    0.456794  11.288717      34.864484        -46.665703   
2021-02-28  36.595259   -3.160455   2.774079      -0.318030         -2.740505   
2021-03-31   0.546488    0.703388   2.218292       0.563964         -1.146959   

            accruals_yr-1  

/var/folders/cy/c49pzcm15cs4xg2nb2d36pkr0000gn/T/ipykernel_4221/1398796258.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tb2 = md2.groupby('date').apply(lm, 'return', *predictor).droplevel(-1, axis=0)


## Model 3

In [10]:
# read the dataset
clct3 = []
rdr3 = pd.read_csv(file3, dtype={'GVKEY': str}, parse_dates=[1], chunksize=100000)
for chunk in tqdm(rdr3):
    chunk = chunk.rename(str.lower, axis=1)
    clct3.append(chunk)
    chunk = None
md3 = pd.concat(clct3, ignore_index=True)
clct3 = None
md3.head()

14it [00:03,  4.58it/s]


,gvkey,date,return,logsize_-1,logb/m_-1,"return_-2,-12","logissues_-1,-36",accruals_yr-1,roa_yr-1,logag_yr-1,"dy_-1,-12","logreturn_-13,-36","logissues_-1,-12","turnover_-1,-12",debt/price_yr-1,sales/price_yr-1
0,1000,1974-04-30,9.5238,1.924431,-0.081561,-0.428572,-0.298542,-0.246,0.0894,0.089507,0.0,-0.551282,-0.153494,9739.004887,1.021711,5.509943
1,1000,1974-05-31,-13.0435,2.016424,0.048045,-0.222223,-0.297521,-0.246,0.0894,0.089507,0.0,-0.686046,-0.150169,9758.016991,0.931915,5.025683
2,1000,1974-06-30,5.0000,1.877682,0.187806,-0.080001,-0.296501,-0.246,0.0894,0.089507,0.0,-0.637681,-0.146841,9405.625892,1.070609,5.773643
3,1000,1974-07-31,-14.2857,1.927492,0.139016,-0.166668,-0.295198,-0.246,0.0894,0.089507,0.0,-0.636363,-0.132863,9020.064092,1.018589,5.493106
4,1000,1974-08-31,16.6667,1.766826,0.293167,-0.322582,-0.301428,-0.246,0.0894,0.089507,0.0,-0.367346,-0.126248,8754.809368,1.196121,6.450510


In [11]:
# preprocessing
# the predictors are winsorized at 0.01 and 0.99 levels
# run cross-sectional reg for every month only the #stocks >= 30
predictor = tuple(md3.columns[3:])
month_list = (
    md3.groupby('date').size().reset_index()
    .pipe(pd.DataFrame.rename, columns={0: 'n_stocks'})
    .pipe(pd.DataFrame.query, "n_stocks >= 30")
    .pipe(lambda x: x['date'].to_list())
)

md3 = (
    md3[md3['date'].isin(month_list)]
)
md3.head()

,gvkey,date,return,logsize_-1,logb/m_-1,"return_-2,-12","logissues_-1,-36",accruals_yr-1,roa_yr-1,logag_yr-1,"dy_-1,-12","logreturn_-13,-36","logissues_-1,-12","turnover_-1,-12",debt/price_yr-1,sales/price_yr-1
0,1000,1974-04-30,9.5238,1.924431,-0.081561,-0.428572,-0.298542,-0.246,0.0894,0.089507,0.0,-0.551282,-0.153494,9739.004887,1.021711,5.509943
1,1000,1974-05-31,-13.0435,2.016424,0.048045,-0.222223,-0.297521,-0.246,0.0894,0.089507,0.0,-0.686046,-0.150169,9758.016991,0.931915,5.025683
2,1000,1974-06-30,5.0000,1.877682,0.187806,-0.080001,-0.296501,-0.246,0.0894,0.089507,0.0,-0.637681,-0.146841,9405.625892,1.070609,5.773643
3,1000,1974-07-31,-14.2857,1.927492,0.139016,-0.166668,-0.295198,-0.246,0.0894,0.089507,0.0,-0.636363,-0.132863,9020.064092,1.018589,5.493106
4,1000,1974-08-31,16.6667,1.766826,0.293167,-0.322582,-0.301428,-0.246,0.0894,0.089507,0.0,-0.367346,-0.126248,8754.809368,1.196121,6.450510


In [12]:
# Fama-MacBeth reg
tb3 = md3.groupby('date').apply(lm, 'return', *predictor).droplevel(-1, axis=0)
#tb3.head(5)
print(pd.concat([tb3.head(5),tb3.tail(5)]))

             constant  logsize_-1  logb/m_-1  return_-2,-12  logissues_-1,-36  \
date                                                                            
1970-01-31  -0.176044   -1.549624   0.973431     -16.298849          4.073901   
1970-02-28 -15.158033    0.723747  -4.495859      -8.525716          4.286630   
1970-03-31   3.957412    0.088445   4.792572      -1.907837         -2.576999   
1970-04-30 -22.475488    2.015282   3.795218      -2.075716          0.334133   
1970-05-31 -10.467963    0.615374   0.533356      -5.296790         -3.958814   
2020-11-30  20.874868   -1.043428   1.691978      -6.547996          4.457705   
2020-12-31  18.833101   -1.436706  -0.383768      -0.256822         -3.491140   
2021-01-31  17.460085   -2.714860  -0.190799       6.307425          2.356019   
2021-02-28  13.817050   -0.488012   2.378251      -2.481078          0.279712   
2021-03-31  -5.428626    0.857520   0.872664       1.804843         -1.064467   

            accruals_yr-1  

/var/folders/cy/c49pzcm15cs4xg2nb2d36pkr0000gn/T/ipykernel_4221/1494935680.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tb3 = md3.groupby('date').apply(lm, 'return', *predictor).droplevel(-1, axis=0)


# Question 2

Time-Series Averages and $t$-Statistic 

In [13]:
tstat(tb1)

,average,t-stat,t-test
constant,1.779453,5.582568,***
logsize_-1,-0.108633,-2.930760,***
logb/m_-1,0.472283,7.565198,***
"return_-2,-12",0.997629,6.626166,***
adj_R^2,0.029237,18.007901,***
#obs,3534.486111,64.091725,***


In [14]:
tstat(tb2)

,average,t-stat,t-test
constant,1.796233,5.608896,***
logsize_-1,-0.101880,-2.888232,***
logb/m_-1,0.391599,6.236552,***
"return_-2,-12",0.859853,5.820084,***
"logissues_-1,-36",-0.197724,-1.795688,*
accruals_yr-1,-0.000822,-1.618250,not sig.
roa_yr-1,1.645383,3.140055,***
logag_yr-1,-0.716954,-5.266677,***
adj_R^2,0.034715,20.154902,***
#obs,2351.054927,77.441085,***


In [15]:
tstat(tb3)

,average,t-stat,t-test
constant,1.664159,5.690504,***
logsize_-1,-0.086585,-2.665991,***
logb/m_-1,0.292799,5.934508,***
"return_-2,-12",0.730241,5.684470,***
"logissues_-1,-36",-0.124036,-1.336839,not sig.
accruals_yr-1,-0.000793,-1.782374,*
roa_yr-1,1.410310,2.662815,***
logag_yr-1,-0.582212,-4.898318,***
"dy_-1,-12",-1.407749,-0.839719,not sig.
"logreturn_-13,-36",-0.057001,-1.030446,not sig.


# Question 3

Forecasting with Rolling Method

In [16]:
# the window size is 10 years, that is, 120 months
mgd = (
    tb3.pipe(pd.DataFrame.drop, columns=['adj_R^2', '#obs'])
    .pipe(lambda x: x.rolling(120, min_periods=120).mean().dropna())
    .pipe(pd.DataFrame.reset_index)
    .pipe(pd.DataFrame.assign,
          date=lambda x: x['date'] + pd.offsets.MonthEnd(1)
         )
    .pipe(pd.merge, md3, how='inner', on='date', suffixes=('_lambda', None))
)
mgd.head()

,date,constant,logsize_-1_lambda,logb/m_-1_lambda,"return_-2,-12_lambda","logissues_-1,-36_lambda",accruals_yr-1_lambda,roa_yr-1_lambda,logag_yr-1_lambda,"dy_-1,-12_lambda",...,"logissues_-1,-36",accruals_yr-1,roa_yr-1,logag_yr-1,"dy_-1,-12","logreturn_-13,-36","logissues_-1,-12","turnover_-1,-12",debt/price_yr-1,sales/price_yr-1
0,1980-01-31,1.160996,-0.082763,0.609285,1.113902,-0.139921,-0.003515,2.537439,-0.008769,-1.604035,...,0.436316,2.457,0.039490,0.215465,0.028814,0.338005,0.131147,21231.256206,0.949495,3.286672
1,1980-01-31,1.160996,-0.082763,0.609285,1.113902,-0.139921,-0.003515,2.537439,-0.008769,-1.604035,...,0.020144,-26.531,0.048066,0.160246,0.064380,-0.013732,0.011390,34601.845097,1.113009,2.718815
2,1980-01-31,1.160996,-0.082763,0.609285,1.113902,-0.139921,-0.003515,2.537439,-0.008769,-1.604035,...,0.658282,-5.146,0.006121,0.637156,0.000000,1.250000,0.000517,1639.790867,3.964470,3.759559
3,1980-01-31,1.160996,-0.082763,0.609285,1.113902,-0.139921,-0.003515,2.537439,-0.008769,-1.604035,...,0.019453,0.378,0.026703,0.115414,0.020000,1.999988,0.000000,42446.134347,1.496344,6.132553
4,1980-01-31,1.160996,-0.082763,0.609285,1.113902,-0.139921,-0.003515,2.537439,-0.008769,-1.604035,...,0.031366,-18.673,0.051809,0.092640,0.084786,-0.169902,0.006524,25583.480753,0.828339,4.417621


In [17]:
res = forecast(mgd, *predictor)
res.head(20)

,date,gvkey,return,forecast
0,1980-01-31,1004,22.6695,0.787708
1,1980-01-31,1010,21.1679,0.835864
2,1980-01-31,1020,-3.5714,0.976677
3,1980-01-31,1025,20.1539,1.507060
4,1980-01-31,1040,-0.8547,0.661331
5,1980-01-31,1043,4.4776,-0.137423
6,1980-01-31,1058,8.6420,0.988632
7,1980-01-31,1059,7.5077,0.752986
8,1980-01-31,1067,-17.3913,0.648664
9,1980-01-31,1070,20.0000,1.060759


# Question 4

Performance and Accuracy

## Accuracy

In [25]:
accuracy_lm=accuracy(res, 'return', 'forecast')
print(accuracy_lm)

          date  accuracy
0   1980-01-31  0.712725
1   1980-02-29  0.330798
2   1980-03-31  0.061590
3   1980-04-30  0.658280
4   1980-05-31  0.756956
..         ...       ...
490 2020-11-30  0.873348
491 2020-12-31  0.746863
492 2021-01-31  0.504244
493 2021-02-28  0.770701
494 2021-03-31  0.725728

[495 rows x 2 columns]


## Performance

In [19]:
# monthly return
mret = performance(res, 'return', 'forecast')
mret

,date,mret
0,1980-01-31,8.620033
1,1980-02-29,-2.341472
2,1980-03-31,-14.888477
3,1980-04-30,4.661579
4,1980-05-31,6.585804
...,...,...
490,2020-11-30,16.324750
491,2020-12-31,8.236453
492,2021-01-31,3.220358
493,2021-02-28,7.638752


In [20]:
# annualized return
annualized = annualret(mret)
annualized

,Year,Annualised Return(%)
0,1980,29.252189
1,1981,5.389230
2,1982,32.471162
3,1983,37.735497
4,1984,-1.771443
5,1985,24.611579
6,1986,10.073370
7,1987,-4.995493
8,1988,21.560339
9,1989,15.452626


In [21]:
min(annualized['Annualised Return(%)'])

-41.66350113958748

In [22]:
annualized['Annualised Return(%)'].mean()

14.989126689171428

# Question 5

In [38]:
accuracy_2000_plus = accuracy_lm[accuracy_lm['date'] > '2000-12-31']

performance_2000_plus = mret[mret['date'] > '2000-12-31']

print("2000 Accuracy Fama-MacBeth Regression")
print(accuracy_2000_plus['accuracy'].mean())

print("2000 Performance Fama-MacBeth Regression")
print(performance_2000_plus['mret'].mean()/100)

2000 Accuracy Fama-MacBeth Regression
0.523579745967432
2000 Performance Fama-MacBeth Regression
0.010111537103181746


In [30]:
print("2000 Accuracy Fama-MacBeth Regression")
print(accuracy_2000_plus['accuracy'].mean())

print("2000 Performance Fama-MacBeth Regression")
print(performance_2000_plus['mret'].mean()*12/100)

2000 Accuracy Fama-MacBeth Regression
0.523579745967432
2000 Performance Fama-MacBeth Regression


KeyError: 'mret'